In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch15
!touch ch15/__init__.py

mkdir: cannot create directory ‘ch15’: File exists


In [0]:
import imp

## 15.0 

- `with` 语句会设置一个临时的上下文，交给上下文管理器对象控制，并且负责清理上下文
- `with` 不仅能管理资源，还能用于去掉常规的设置和清理代码，或者在另一个过程前后执行的操作

## 15.1 先做这个，再做那个： `if` 语句之外的 `else` 块

- `else` 子句不仅能在 `if` 语句中使用，还能在 `for`、 `while` 和 `try` 语句中使用
- `for`
  - 仅当 `for` 循环执行完毕时 (即 `for` 循环没有被 `break` 语句中止) 才运行 `else` 块
- `while`
  - 仅当 `while` 循环因为条件为假值而退出时 (即 `while` 循环没有被 `break` 语句中止) 才运行 `else` 块
- `try`
  - 仅当 `try` 块中没有异常抛出时才运行 `else` 块
  - `else` 子句抛出的异常不会由前面的 `except` 子句处理
- 如果异常或者 `return`、`break` 或 `continue` 语句导致控制权跳到了复合语句的主块之外，`else` 子句也会被跳过

###### `for` 循环示例

```python
for item in my_list:
  if item.flavor == 'banana':
    break
else:
  reaise ValueError('No banana flavor found!')
```

###### `try/except` 示例

- 为了清晰和准确，`try` 块中应该只包含抛出预期异常的语句

```python
try:
  dangerous_call()
except OSError:
  log('OSError...')
else:
  after_call()
```

- 只有 `try` 块不抛出异常，才会执行 `after_call()`

###### EAFP 与 LBYL

- EAFP
  - 取得原谅比获得许可容易（easier to ask for forgiveness than permission）
  - 这是一种常见的 Python 编程风格，先假定存在有效的键或属性，如果假定不成立，那么捕获异常
  - 这种风格简单明快，特点是代码中有很多 `try` 和 `except` 语句

- LBYL
  - 三思而后行（look before you leap）
  - 此编程风格在调用函数或查找属性或键之前显式测试前提条件
  - 这种风格的特点是代码中有很多 `if` 语句
  - 对 `if key in mapping: return mapping[key]` 这段代码来说
    - 如果在测试之后，但在查找之前，另一个线程从映射中删除了那个键，那么这段代码就会失败
    - 这个问题可以使用锁或者 EAFP 风格解决

## 15.2 上下文管理器和 `with` 块

- 上下文管理器对象存在的目的是管理 `with` 语句，就像迭代器的存在是为了管理 `for` 语句一样
- `with` 语句的目的是简化 `try/finally` 模式
  - 这种模式用于保证一段代码运行完毕后执行某项操作
    - 即便那段代码由于异常、`return` 语句或 `sys.exit()` 调用而中止，也会执行指定的操作
  - `finally` 子句中的代码通常用于释放重要的资源，或者还原临时变更的状态
- 上下文管理器协议包含 `__enter__` 和 `__exit__` 两个方法
  - `with` 语句开始运行时，会在上下文管理器对象上调用 `__enter__` 方法
  - `with` 语句运行结束后，会在上下文管理器对象上调用 `__exit__` 方法，以此扮演 `finally` 子句的角色

###### 示例 15-1 演示把文件对象当成上下文管理器使用

In [0]:
with open('ch15/mirror.py') as fp:  # fp 绑定到打开的文件上，因为文件的 __enter__ 方法返回 self
  src = fp.read(60)

In [0]:
len(src)

60

In [0]:
fp  # fp 变量仍然可用

<_io.TextIOWrapper name='ch15/mirror.py' mode='r' encoding='UTF-8'>

In [0]:
fp.closed, fp.encoding  # 可以读取 fp 对象的属性

(True, 'UTF-8')

In [0]:
fp.read(60)  # 不能在 fp 上执行 I/O 操作，因为在 with 块的末尾，调用 TextIOWrapper.__exit__ 方法把文件关闭了

ValueError: ignored

###### 示例 15-3　mirror.py：LookingGlass 上下文管理器类的代码

In [0]:
%%writefile ch15/mirror.py
class LookingGlass:

  def __enter__(self):
    import sys
    self.original_write = sys.stdout.write
    sys.stdout.write = self.reverse_write
    return 'JABBERWOCKY'  # 存入到目标变量 what 中

  def reverse_write(self, text):
    self.original_write(text[::-1])

  def __exit__(self, exc_type, exc_value, traceback):
    import sys 
    sys.stdout.write = self.original_write  # 还原为原来的输入方法
    if exc_type is ZeroDivisionError:
      print('Please DO NOT divide by zero!')
    return True 

Overwriting ch15/mirror.py


###### 示例 15-2　测试 `LookingGlass` 上下文管理器类

In [0]:
import ch15.mirror
imp.reload(ch15.mirror)
from ch15.mirror import LookingGlass

In [0]:
with LookingGlass() as what:
  print('Alice, Kitty and Snowdrop')
  print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [0]:
print(what)

JABBERWOCKY


In [0]:
print('Back to normal.')

Back to normal.


###### 传给 `__enter__` 和 `__exit__` 方法的参数

- 解释器调用 `__enter__` 方法时，除了隐式的 `self` 之外，不会传入任何参数
- 传给 `__exit__` 方法的三个参数列举
  1. `exc_type`
    - 异常类
  2. `exc_value`
    - 异常实例。有时会有参数传给异常构造方法，例如错误消息，这些参数可以使用 `exc_value.args` 获取
  3. `traceback`
    - `traceback` 对象

###### 示例 15-4　在 `with` 块之外使用 `LookingGlass` 类

- 直接在 colab 中输出无效，必须要 `print` 打印才有效

In [0]:
from ch15.mirror import LookingGlass
manager = LookingGlass()
manager

In [0]:
monster = manager.__enter__()

In [0]:
print(monster == 'JABBERWOCKY')

eurT


In [0]:
print(monster)

YKCOWREBBAJ


In [0]:
print(manager)

>0ad176deb6f7x0 ta tcejbo ssalGgnikooL.rorrim.51hc<


In [0]:
print(manager.__exit__(None, None, None))

True


In [0]:
print(monster)

JABBERWOCKY


## 15.3 `contextlib` 模块中的实用工具

- `closing`
  - 如果对象提供了 `close()` 方法，但没有实现 `__enter__/__exit__` 协议，那么可以使用这个函数构建上下文管理器
- `suppress`
  - 构建临时忽略指定异常的上下文管理器
- `@contextmanager`
  - 这个装饰器把简单的生成器函数变成上下文管理器，这样就不用创建类去实现管理器协议了
- `ContextDecorator`
  - 基类，用于定义基于类的上下文管理器。这种上下文管理器也能用于装饰函数，在受管理的上下文中运行整个函数
- `ExitStack`
  - 这个上下文管理器能进入多个上下文管理器
  - `with` 块结束时，`ExitStack` 按照后进先出的顺序调用栈中各个上下文管理器的 `__exit__` 方法
  - 如果事先不知道 `with` 块要进入多少个上下文管理器，可以使用这个类
    - 例如，同时打开任意一个文件列表中的所有文件

## 15.4 使用 `@contextmanager`

- `@contextmanager` 装饰器能减少创建上下文管理器的样板代码量 
  - 因为不用编写一个完整的类，定义 `__enter__` 和 `__exit__` 方法
  - 只需实现有一个 `yield` 语句的生成器，生成想让 `__enter__` 方法返回的值
- 在使用 `@contextmanager` 装饰的生成器中，`yield` 语句的作用是把函数的定义体分成两部分
  - `yield` 语句前面的所有代码在 `with` 块开始时（即解释器调用 `__enter__` 方法时）执行
  - `yield` 语句后面的代码在 `with` 块结束时（即调用 `__exit__` 方法时）执行
- 使用 `@contextmanager` 装饰器时，要把 `yield` 语句放在 `try/finally` 语句中（或者放在 `with` 语句中）
  - 这是无法避免的，因为我们永远不知道上下文管理器的用户会在 with 块中做什么
- `@contextmanager` 装饰器优雅且实用，把三个不同的 Python 特性结合到了一起：函数装饰器、生成器和 `with` 语句


###### 示例 15-5　mirror_gen.py：使用生成器实现的上下文管理器

In [0]:
%%writefile ch15/mirror_gen.py
import contextlib


@contextlib.contextmanager
def looking_glass():
  import sys
  original_write = sys.stdout.write

  def reverse_write(text):
    original_write(text[::-1])

  sys.stdout.write = reverse_write
  yield 'JABBEREOCKY'
  sys.stdout.write = original_write

Writing ch15/mirror_gen.py


In [0]:
from ch15.mirror_gen import looking_glass
with looking_glass() as what:
  print('Alice, Kitty and Snowdrop')
  print(what)

pordwonS dna yttiK ,ecilA
YKCOEREBBAJ


In [0]:
print(what)

JABBEREOCKY


###### 示例 15-7　mirror_gen_exc.py：基于生成器的上下文管理器，而且实现了异常处理——从外部看，行为与示例 15-3 一样

In [0]:
%%writefile ch15/mirror_gen_exc.py
import contextlib


@contextlib.contextmanager
def looking_glass():
  import sys
  original_write = sys.stdout.write

  def reverse_write(text):
    original_write(text[::-1])

  sys.stdout.write = reverse_write
  msg = ''
  try:
    yield 'JABBEREOCKY'
  except ZeroDivisionError:
    msg = 'Please DO NOT divide by zero!'
  finally:
    sys.stdout.write = original_write
    if msg:
      print(msg)

Overwriting ch15/mirror_gen_exc.py


In [0]:
import ch15.mirror_gen_exc
imp.reload(ch15.mirror_gen_exc)
from ch15.mirror_gen_exc import looking_glass

In [0]:
with looking_glass() as what:
  print('Alice, Kitty and Snowdrop')
  print(what)

pordwonS dna yttiK ,ecilA
YKCOEREBBAJ


In [0]:
print(what)

JABBEREOCKY
